In [260]:
from prisma import Prisma
import numpy as np
db = Prisma()
db.connect()

sessions = db.session.find_many(
  where={
    'OR': [
      {'moveId': 'cllsr98nl0000m3wg0pqpvjuy'},
      {'moveId': 'cllsr98nn0002m3wg3l6nbb5l'},
      {'moveId': 'cllsr98nn0004m3wgcm0bpdta'},
      {'moveId': 'cllsr98no0006m3wgbau6ed9q'},
    ]
  },
      # 'moveId': ['cllsr98nl0000m3wg0pqpvjuy','cllsr98nn0002m3wg3l6nbb5l']
  include={
    'move':True
  }
)
sessions[:] = [i for i in sessions if len(i.dict()['accelerationX']) == len(i.dict()['gyroX'])]
len(sessions)
for s in sessions: 
  print(s.dict()['move']['name'], len(s.dict()['gyroX']), len(s.dict()['accelerationX']))


db.disconnect()


Down Up 87 87
Down Up 78 78
Down Up 72 72
Down Up 84 84
Down Up 72 72
Down Up 96 96
Down Up 78 78
Down Up 96 96
Square 134 134
Square 129 129
Square 119 119
Square 125 125
Square 171 171
Circle 77 77
Square 113 113
Square 132 132
Square 149 149
Circle 95 95
Circle 74 74
Circle 72 72
Circle 71 71
Circle 72 72
Circle 66 66
Up Down 78 78
Up Down 84 84
Circle 105 105
Circle 74 74
Up Down 66 66
Up Down 86 86
Up Down 84 84
Up Down 86 86
Up Down 98 98
Up Down 167 167


In [261]:
import pandas

data_columns = ['gyroX','gyroY','gyroZ','accelerationX','accelerationY','accelerationZ']

def prep_sessions(sessions,data_columns):
  #convert each session to a an array of just the data
  dfs = []
  for s in sessions:
    # extract out just data_columns keys from s
    d = {}
    for k, v in s:
      if k in data_columns:
        d[k] = s.dict()[k]
        
    df = pandas.DataFrame(d)[data_columns]
    np_array = df.to_numpy().astype(float)
    dfs.append(np_array)
  
  return dfs

def generate_labels_from_sessions(sessions):
  names = []
  for s in sessions:
    names.append(s.dict()['move']['name'])
  
  # convert names to list of numbers, kinda like an enum, each uniqe string becomes a nubmer
  labels_key = {}
  v = -1
  labels = []
  for n in names:
    if n not in labels_key:
      v += 1
      labels_key[n] = v
    labels.append(labels_key[n])
    
  return labels, labels_key

def vertically_scale_array(original_array):
  # scale it up vertically
  original_shape = original_array.shape
  target_shape = (200, original_shape[1])

  # Generate indices for original and target arrays
  original_indices = np.arange(original_shape[0])
  target_indices = np.linspace(0, original_shape[0] - 1, target_shape[0])

  scaled_up_array = np.empty(target_shape)

  for col in range(original_shape[1]):
    scaled_up_array[:, col] = np.interp(target_indices, original_indices, original_array[:, col])
      
  return scaled_up_array


session_arrays = prep_sessions(sessions, data_columns)
normalized_session_arrays = [vertically_scale_array(s) for s in session_arrays]
labels, labels_key = generate_labels_from_sessions(sessions)
labels_key
 

{'Down Up': 0, 'Square': 1, 'Circle': 2, 'Up Down': 3}

In [215]:
import tensorflow as tf
from tensorflow.keras import layers

# Stack the arrays
data = np.stack(normalized_session_arrays)  # 33 arrays of shape (200, 6)
labels = np.array(labels)  # 33 labels

model = models.Sequential()
model.add(layers.Input(shape=(200, 6)))  # Input layer

# Add some layers (adjust as needed)
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(33, activation='softmax')) 
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(data, one_hot_labels, epochs=10, batch_size=32, validation_split=0.2)

model.fit(x=data, y=labels, epochs=10)

33
[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 2, 2, 2, 2, 2, 3, 3, 2, 2, 3, 3, 3, 3, 3, 3]
Epoch 1/10


2/2 [==============================] - 1s 47ms/step - loss: 5.5150 - accuracy: 0.3636
Epoch 2/10
2/2 [==============================] - 0s 42ms/step - loss: 13.2549 - accuracy: 0.8182
Epoch 3/10
2/2 [==============================] - 0s 49ms/step - loss: 1.7315 - accuracy: 0.9394
Epoch 4/10
2/2 [==============================] - 0s 46ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 5/10
2/2 [==============================] - 0s 45ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 6/10
2/2 [==============================] - 0s 40ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 7/10
2/2 [==============================] - 0s 43ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 8/10
2/2 [==============================] - 0s 63ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 9/10
2/2 [==============================] - 0s 46ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 10/10
2/2 [==============================] - 0s 53ms/step - loss: 0.0000e+00 - accuracy: 1.0000


In [255]:
db = Prisma()
db.connect()

sessions = db.session.find_many(
  where={
    'moveId': 'cllpzabyd0000m35m17byo2dp'
  },
  include={
    'move':True
  }
)
sessions[:] = [i for i in sessions if len(i.dict()['accelerationX']) == len(i.dict()['gyroX'])]
len(sessions)
for s in sessions: 
  print(s.dict()['move']['name'], len(s.dict()['gyroX']), len(s.dict()['accelerationX']))


db.disconnect()

Shoot In 93 93
Shoot In 89 89
Shoot In 99 99


In [259]:
session_arrays = prep_sessions(sessions, data_columns)
normalized_session_arrays_test = [vertically_scale_array(s) for s in session_arrays]
print(labels_key)
model(np.array([normalized_session_arrays_test[2]]))

{'Down Up': 0, 'Square': 1, 'Circle': 2, 'Up Down': 3}


<tf.Tensor: shape=(1, 4), dtype=float32, numpy=array([[153.58553 , -48.847153, -45.16167 , -60.9632  ]], dtype=float32)>